# Projet Accidents corporels de la circulation routière

## Auteur : Nicolas Lejay

## Juin 2023

***
### A propos
- Ce projet est réalisé à partir de fichiers regroupant les informations sur les accidents corporels de la circulation routière entre 2005 et 2021. L'Observatoire National Interministérielle de la Sécurité Routière fournit quatre fichiers par année : sur les caractéristiques de l'accident, sur le lieu, sur les véhicules impliqués et enfin sur les personnes impliquées. Ici, nous utiliserons quatres fichiers qui sont la compilations de l'ensemble des années. Ces fichiers ont également été nettoyés.
- Pour toute information sur le jeu de données, la signification des variablmes ainsi que sur la définition donnée à « accident corporel de la circulation routière », se repporter au fichier README.md et à la notice de l'ONISR fournis dans le repository : https://github.com/nlejay/accidents_circulation

***
# 1. Importation des fichiers

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import colorcet as cc

# Changement de police des graphiques
plt.rc('font', family = 'serif', serif = 'cmr10')
plt.rcParams.update({"text.usetex": True, "axes.formatter.use_mathtext" : True})

In [2]:
df_caract = pd.read_csv('src/caracteristiques_net_19_06_23.csv',
                        index_col=0,
                        # colonnes avec plusieurs types de formats converties au format str
                        dtype={'hrmn': 'str', 'com' : 'str', 'lat':'str', 'long':'str', 'dep':'str', 'gps':'str'})

df_lieux = pd.read_csv('src/lieux_net_19_06_23.csv',
                        index_col=0,
                        # colonnes avec plusieurs types de formats converties au format str
                        dtype={'voie': 'str', 'pr' : 'str', 'pr1':'str', 'larrout':'str', 'lartpc':'str'})

df_usagers = pd.read_csv('src/usagers_net_19_06_23.csv',
                         index_col=0,
                         # colonnes avec plusieurs types de formats converties au format str
                         dtype={'actp':'str', 'id_vehicule':'str'})
                         
df_veh = pd.read_csv('src/vehicules_net_19_06_23.csv',
                     index_col=0,
                     # colonnes avec plusieurs types de formats converties au format str
                     dtype={'id_vehicule':'str'})

**Dataframe *df_caract***

In [3]:
df_caract.head()

,Num_Acc,jour,mois,an,hrmn,lum,dep,com,agg,int,atm,col,adr,lat,long
0,201900000001,30,11,2019,01:30,4.0,93,93053,1,1.0,1.0,2.0,AUTOROUTE A3,48.8962100,2.4701200
1,201900000002,30,11,2019,02:50,3.0,93,93066,1,1.0,1.0,6.0,AUTOROUTE A1,48.9307000,2.3688000
2,201900000003,28,11,2019,15:15,1.0,92,92036,1,1.0,1.0,4.0,AUTOROUTE A86,48.9358718,2.3191744
3,201900000004,30,11,2019,20:20,5.0,94,94069,1,1.0,1.0,4.0,A4,48.8173295,2.4281502
4,201900000005,30,11,2019,04:00,3.0,94,94028,1,1.0,1.0,2.0,A86 INT,48.7763620,2.4332540


In [4]:
df_caract.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1121571 entries, 0 to 1121570
Data columns (total 15 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   Num_Acc  1121571 non-null  int64  
 1   jour     1121571 non-null  int64  
 2   mois     1121571 non-null  int64  
 3   an       1121571 non-null  int64  
 4   hrmn     1121571 non-null  object 
 5   lum      1121566 non-null  float64
 6   dep      1121571 non-null  object 
 7   com      1121569 non-null  object 
 8   agg      1121571 non-null  int64  
 9   int      1121463 non-null  float64
 10  atm      1121477 non-null  float64
 11  col      1120015 non-null  float64
 12  adr      978295 non-null   object 
 13  lat      634503 non-null   object 
 14  long     634499 non-null   object 
dtypes: float64(4), int64(5), object(6)
memory usage: 136.9+ MB


**Dataframe *df_lieux***

In [ ]:
df_lieux.head()

In [ ]:
df_lieux.info()

**dataframe *df_usagers***

In [ ]:
df_usagers.head()

In [ ]:
df_usagers.info()

On remarque plus d'observations que dans les deux dataframes précédents. C'est normal : un accident peut impliquer une ou plusieurs personnes.

**Dataframe *df_veh***

In [ ]:
df_veh.head()

In [ ]:
df_veh.info()

Là-aussi, un accident implique souvent plusieurs véhicules. Nous avons donc plus de lignes dans ce dataframe que dans celui sur les accidents.

***
# 2.Étude temporelle
## 2.1. Évolution du nombre d'accidents, de morts et de blessés chaque année
### 2.1.1. Évolution du nombre et variation en pourcentage

1 121 571 accidents répertoriés sur 17 ans, soit une moyenne de 65975 par an. Mais comment a évolué le nombre d'accidents corporels sur la période 2005-2017 ? Et comment ont évolué le nombre de morts et de blessés ?

In [ ]:
# Calcul du nombre d'accidents par an
acc_par_an = (df_caract
              .groupby('an')
              .agg(Effectif = pd.NamedAgg('Num_Acc','count'))
              .reset_index())

#Calcul de la variation annuelle
acc_par_an['var_pourc'] = acc_par_an['Effectif'].pct_change() * 100

acc_par_an.transpose()

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1,figsize=(8,6))
ax1.plot(acc_par_an['an'], acc_par_an['Effectif'], color="tab:blue")
ax1.set_ylim(0,90000)
ax1.set_xticks([i for i in range(2005,2022,2)], [i for i in range(2005,2022,2)])
ax1.set_xlabel('Année', size=12)
ax1.set_ylabel("Nombre d'accidents", size=12)
ax1.set_title("Evolution du nombre d'accidents corporels de la circulation routière sur la période 2005-2021", size=12, y=1.02)
ax1.grid(visible=True, axis='y', color="#e0e0e0")


couleurs = ["crimson" if acc_par_an.iloc[i,2]>0 else "darkgreen" for i in range(len(acc_par_an))]
ax2.bar(acc_par_an['an'], acc_par_an['var_pourc'], color=couleurs)
ax2.set_xlabel('Année', size=12)
ax2.set_xlim(2004,2022)
ax2.set_ylabel("Variation en pourcentage", size=12)
ax2.set_ylim(-20,20)
ax2.set_xticks([i for i in range(2005,2022,2)], [i for i in range(2005,2022,2)])
ax2.set_yticks([i for i in range(-20,25,5)], [i for i in range(-20,25,5)])
ax2.grid(visible=True, axis='y', color="#e0e0e0")
ax2.set_title("Variation annuelle en pourcentage du nombre d'accidents corporels de la circulation routière sur la période 2005-2021", size=12, y=1.02)
ax2.set_axisbelow(True)
ax2.hlines(0,2004,2022, color='black', linewidth=0.5)

sns.despine()
plt.tight_layout()
plt.show()

On observe une baisse de 2005 à 2013 puis une stabilisation autour de 60 000 accidents par an. Pour l'année 2020, la chute est sûrement due aux confinents pendant l'épidémie de COVID-19. On remarque d'ailleurs une hausse en 2021 qui ramène quasiment au niveau pré-2020.

Concentrons-nous maintenant sur le nombre de morts et de blessés. Pour cela, une jointure entre *df_caract* et *df_usagers* est nécessaire cr l'année n'est renseignée que dans *df_caract*.

In [ ]:
# Calcul du nombre de morts et blessés de chaque type chaque année
gravite_par_an = (df_usagers[['Num_Acc','grav']]
                  .merge(df_caract[['Num_Acc', 'an']], how='inner', left_on='Num_Acc', right_on='Num_Acc')
                  .pivot_table(index='an', columns='grav', values='Num_Acc', aggfunc='count')
                  .rename(columns={1:'Indemne',
                                    2:'Tué',
                                    3:'Hospitalisé',
                                    4:'Bléssé léger'})
                 )

gravite_par_an.transpose().head()

In [ ]:
# Calcul de la variation annuelle en pourcentage
var_gravite_par_an = round(gravite_par_an.pct_change()*100, 1)
var_gravite_par_an.transpose()

In [ ]:
grav_par_an_long = gravite_par_an.unstack().reset_index().rename(columns={0:'Effectif'})
morts_par_an = grav_par_an_long.loc[grav_par_an_long['grav']=='Tué']
pd.options.mode.chained_assignment = None
morts_par_an['var_pourc'] = morts_par_an['Effectif'].pct_change()*100

fig, (ax1, ax2) = plt.subplots(2,1,figsize=(8,6))
ax1.plot(morts_par_an['an'], morts_par_an['Effectif'], color="tab:blue")
ax1.set_ylim(0,6000)
ax1.set_xticks([i for i in range(2005,2022,2)], [i for i in range(2005,2022,2)])
ax1.set_xlabel('Année', size=12)
ax1.set_ylabel("Nombre de morts", size=12)
ax1.set_title("Evolution du nombre de morts d'un accident de la circulation routière sur la période 2005-2021", size=12, y=1.02)
ax1.grid(visible=True, axis='y', color="#e0e0e0")


couleurs = ["crimson" if morts_par_an.iloc[i,3]>0 else "darkgreen" for i in range(len(morts_par_an))]
ax2.bar(morts_par_an['an'], morts_par_an['var_pourc'], color=couleurs)
ax2.set_xlabel('Année', size=12)
ax2.set_xlim(2004,2022)
ax2.set_ylabel("Variation en pourcentage", size=12)
ax2.set_ylim(-25,20)
ax2.set_xticks([i for i in range(2005,2022,2)], [i for i in range(2005,2022,2)])
ax2.set_yticks([i for i in range(-20,25,5)], [i for i in range(-20,25,5)])
ax2.grid(visible=True, axis='y', color="#e0e0e0")
ax2.set_title("Variation annuelle en pourcentage du nombre de morts d'accidents de la circulation routière sur la période 2005-2021", size=12, y=1.02)
ax2.set_axisbelow(True)
ax2.hlines(0,2004,2022, color='black', linewidth=0.5)

sns.despine()
plt.tight_layout()
plt.show()

In [ ]:
hosp_par_an = grav_par_an_long.loc[grav_par_an_long['grav']=='Hospitalisé']
hosp_par_an['var_pourc'] = hosp_par_an['Effectif'].pct_change()*100

fig, (ax1, ax2) = plt.subplots(2,1,figsize=(8,6))
ax1.plot(hosp_par_an['an'], hosp_par_an['Effectif'], color="tab:blue")
ax1.set_ylim(0,42000)
ax1.set_xticks([i for i in range(2005,2022,2)], [i for i in range(2005,2022,2)])
ax1.set_xlabel('Année', size=12)
ax1.set_ylabel("Nombre de blessés hospitalisés", size=12)
ax1.set_title("Evolution du nombre de blessés hospitalisés suite à un accident de la circulation routière sur la période 2005-2021", size=12, y=1.02)
ax1.grid(visible=True, axis='y', color="#e0e0e0")


couleurs = ["crimson" if hosp_par_an.iloc[i,3]>0 else "darkgreen" for i in range(len(hosp_par_an))]
ax2.bar(hosp_par_an['an'], hosp_par_an['var_pourc'], color=couleurs)
ax2.set_xlabel('Année', size=12)
ax2.set_xlim(2004,2022)
ax2.set_ylabel("Variation en pourcentage", size=12)
ax2.set_ylim(-20,20)
ax2.set_xticks([i for i in range(2005,2022,2)], [i for i in range(2005,2022,2)])
ax2.set_yticks([i for i in range(-25,25,5)], [i for i in range(-25,25,5)])
ax2.grid(visible=True, axis='y', color="#e0e0e0")
ax2.set_title("Variation annuelle en pourcentage du nombre de blesssés hospitalisés suite à un accident de la circulation routière sur la période 2005-2021", size=12, y=1.02)
ax2.set_axisbelow(True)
ax2.hlines(0,2004,2022, color='black', linewidth=0.5)

sns.despine()
plt.tight_layout()
plt.show()

In [ ]:
df_var_pourc = pd.concat([acc_par_an.set_index('an')['var_pourc'], morts_par_an.set_index('an')['var_pourc'], hosp_par_an.set_index('an')['var_pourc']], axis=1, keys=['acc', 'morts', 'hosp'])
df_var_pourc = (df_var_pourc
                .stack()
                .reset_index(level=1)
                .rename(columns ={'level_1':'categorie', 0:'var_pourc'})
               )
df_var_pourc.head()

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))

sns.barplot(x='an', y='var_pourc', data=df_var_pourc.reset_index(), hue='categorie', ax=ax)
plt.hlines(0,-1,17, color='black', linewidth=0.5)
plt.title("Comparaison des variations annuelles du nombre d'accidents, de morts et d'hospitalisés", size=12)
plt.xlabel("Année", size=12)
plt.ylabel("Variation en pourcentage", size=12)
plt.grid(visible=True, axis='y', color="#e0e0e0")
ax.set_axisbelow(True)
plt.show()

Le nombre de blessés hospitalisés et le nombre de morts ont à peu près suivi la même évolution que le nombre d'accidents. On remarque néanmoins une nette différence en 2018 et 2019 entre la variation du nombre d'accidents et celle du nombre d'hospitalisés.


On peut vérifier s'il y a corrélation linéaire entre nombre de morts (resp. d'hospitalisés) et nombre d'accidents. Ce n'est pas a priori évident car les améliorations d'équipements de sécurité des véhicules ainsi que des aménagements routiers pourraient faire chuter le nombre de morts ou de blessés graves plus vite que le nombre d'accidents.

### 2.1.2. Y-a-t-il eu corélation entre nombre de morts (resp. de blessés hospitalisés) et le nombre d'accidents entre 2005 et 2021 ?

- **Corrélation nombre de morts/nombre d'accidents**

In [ ]:
morts_vs_acc = pd.DataFrame(acc_par_an['Effectif']).rename(columns={'Effectif':'Accidents'})
morts_vs_acc['Morts'] = morts_par_an['Effectif'].reset_index(drop=True)

fig, ax = plt.subplots(figsize=(6,4))

sns.regplot(x='Accidents', y='Morts', data=morts_vs_acc, ci=None, line_kws={'color':'red'})
plt.xlabel("Nombre d'accidents", size=12)
plt.ylabel("Nombre de morts", size=12)
plt.title("Nombre de morts suivant le nombre d'accidents corporels de la circulation routière", size=12)

plt.show()

Il semble y avoir une corrélation linéaire entre les deux variables. Regardons la normalité des deux variables.

In [ ]:
import pingouin

fig, ((ax1, ax2)) = plt.subplots(1,2, figsize=(10,4))
pingouin.qqplot(morts_vs_acc['Accidents'], dist='norm', color='tab:blue', ax=ax1)
ax1.set_title('Morts')
pingouin.qqplot(morts_vs_acc['Morts'], dist='norm', color='tab:blue', ax=ax2)
ax2.set_title('Accidents')
plt.suptitle('Comparaison de la distribution des variables à une distribution normale')
fig.tight_layout()
plt.show()

Les variables ne suivent pas une distribution normale. Nous allons faire un test de corrélation de Spearman :
- Hypothèse nulle $H_0$ : pas de corrélation entre les variables (coefficient de corrélation nul)
- Hypothèse alternative $H_1$ : corrélation positive (coefficient de corrélation positif et significativement différent de 0) -> test unilatéral
- Seuil de signification $\alpha=0,05$

In [ ]:
pingouin.corr(morts_vs_acc['Morts'], morts_vs_acc['Accidents'], alternative='greater', method='spearman')

Le test de Spearman indique que les deux variables sont significativement corrélées ($\small p<0.0001^{***}$). La corrélation est très forte : $\small\rho = 0,97$.

- **Corrélation nombre d'hospitalisés/nombre d'accidents**

In [ ]:
# Dataframe hospitalisés vs accidents
hosp_vs_acc = pd.DataFrame(acc_par_an['Effectif']).rename(columns={'Effectif':'Accidents'})
hosp_vs_acc['Hospitalisés'] = hosp_par_an['Effectif'].reset_index(drop=True)

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))

sns.regplot(x=acc_par_an['Effectif'], y=hosp_par_an['Effectif'], ci=None, line_kws={'color':'red'})
plt.xlabel("Nombre d'accidents", size=12)
plt.ylabel("Nombre d'hospitalisés", size=12)
plt.title("Nombre de blessés hospitalisés suivant le nombre d'accidents corporels de la circulation routière", size=12)

plt.show()

Il semble y avoir une bonne corrélation positive entre les deux variables. On sait que le nombre d'accidents ne suit pas une distribution normale donc faisons un test de corrélation de Spearman :

- Hypothèse nulle $H_0$ : pas de corrélation entre les variables (coefficient de corrélation nul)
- Hypothèse alternative $H_1$ : corrélation positive (coefficient de corrélation positif et significativement différent de 0) -> test unilatéral
- Seuil de signification $\alpha=0,05$

In [ ]:
pingouin.corr(hosp_vs_acc['Hospitalisés'], hosp_vs_acc['Accidents'], alternative='greater', method='spearman')

Le test de Spearman indique que les deux variables sont significativement corrélées ($\small p<0.0001^{***}$). La corrélation est très forte : $\small\rho = 0,96$.

### 2.1.2. Les proportions de blessés légers, hospitalisés et morts restent-ils constants au fil des années ?

In [ ]:
pourc_par_grav = gravite_par_an.divide(gravite_par_an.sum(axis=1), axis=0)*100
pourc_par_grav

In [ ]:
fig,ax = plt.subplots(figsize=(8,4))
plt.stackplot(pourc_par_grav.index, pourc_par_grav.transpose(), labels=pourc_par_grav.columns, cmap='tab10')

#Mise en forme des axes, titre et légende
plt.title('Evolution du pourcentage de blessés de chaque type parmi les personnes impliquées dans un accident corporel de la circulation', fontsize=14)
plt.xlabel('Année', fontsize=12)
plt.ylabel('Pourcentage', fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(title='Type de blessé', bbox_to_anchor=[1.3,1], fontsize=10, title_fontsize=11)
plt.show()

Les proportions restent à peu près les mêmes à part une baisse du pourcentage d'hospitalisés au profit du pourcentage de blessés légers entre 2017 et 2019.

## 2.2 Existe-t-il des mois plus accidentogènes que d'autres ?

### 2.2.1. Calcul du nombre moyen d'accidents pour chaque mois de l'année sur la période 2005-2017

L'année 2020 étabnt particulière avec des mois de confinement, elle est retirée de cette partie de l'étude.

In [ ]:
# Nombre d'accidents par année et par mois
acc_par_mois = (df_caract.loc[df_caract['an']!=2020]
                .groupby(['an','mois'])
                .agg(Effectif = pd.NamedAgg('Num_Acc','count'))
                .reset_index())
acc_par_mois.head()

In [ ]:
acc_par_mois['Effectif'].describe()

In [ ]:
fig, ax= plt.subplots(figsize=(6,4))
sns.histplot(x='Effectif', data=acc_par_mois, bins=40)
plt.xlabel("Nombre d'accidents")
plt.ylabel("Nombre de mois")
plt.title("Distribution du nombre d'accidents mensuel")
plt.show()

In [ ]:
# Moyenne par mois
moy_par_mois = (acc_par_mois
                .groupby('mois')
                .agg(moyenne = pd.NamedAgg('Effectif','mean'),
                     mediane = pd.NamedAgg('Effectif','median'),
                     variance = pd.NamedAgg('Effectif','var'),
                     std = pd.NamedAgg('Effectif','std'))
                .reset_index()
               )
moy_par_mois

In [ ]:
# Représentation graphique du nombre d'accidents moyen par mois de l'année

fig, ax = plt.subplots(figsize=(8,4))
sns.barplot(x='mois', y='Effectif', data= acc_par_mois, color='tab:blue', errorbar='sd')
plt.ylim(0,8000)
plt.xlabel('Mois', size=12)
plt.ylabel("Nombre moyen d'accidents", size=12)
plt.xticks([i for i in range(12)], ["Janv", "Fév", "Mars", "Avr", "Mai", "Juin", "Juil", "Août", "Sept", "Oct", "Nov", "Déc"], rotation=45)
plt.grid(visible=True, axis='y', color="#e0e0e0")
plt.title("Nombre moyen d'accidents et écart-type pour chaque mois de l'année sur la période 2005-2021 (hors 2020)", size=12)
ax.set_axisbelow(True)
plt.show()

In [ ]:
# boxplot du nombre d'accidents par mois de l'année

fig, ax = plt.subplots(figsize=(8,4))
sns.boxplot(x='mois', y='Effectif', data= acc_par_mois, color='tab:blue', whis=[0,100])
plt.xlabel('Mois', size=12)
plt.ylabel("Nombre d'accidents", size=12)
plt.xticks([i for i in range(12)], ["Janv", "Fév", "Mars", "Avr", "Mai", "Juin", "Juil", "Août", "Sept", "Oct", "Nov", "Déc"], rotation=45)
plt.grid(visible=True, axis='y', color="#e0e0e0")
plt.title("Diagrammes en boîte du nombre d'accidents pour chaque mois de l'année sur la période 2005-2021 (hors 2020)", size=12)
ax.set_axisbelow(True)
plt.show()

On remarque que les mois de juin, septembre et octobre ont été les plus accidentogènes, février étant le mois le moins accidentogène. Mais ces différence sont-elles significatives ? Pour répondre, nous pouvons voir s'il est possible de réaliser une ANOVA.

### 2.2.2. ANOVA

In [ ]:
#Définition d'une fonction d'affichage de graphiques de diagnostics d'un modèle

def diagnostic_plots(fitted, resid, norm_resid, leverage, alpha=0.5):
    """Fonction qui affiche les graphiques de diagnostic d'un modèle:
    - residus vs valeurs prédites
    - qqplot résidus vs distribution normale
    - scale location
    - résidus vs effet de levier
    
    paramètres :
    - fitted : valeurs prédites
    - resid : résidus
    - norm_resid : résidus normalisés
    - leverage : effet de levier
    - alpha : opacité des points (défaut=0.5)
    """
    
    #racine carrée des résidus standardisés
    resid_sqrt = np.sqrt(np.abs(norm_resid))
    
    # Graphiques de diagnostic d'un
    fig,((ax00, ax01), (ax10,ax11)) = plt.subplots(2,2,figsize=(7,7))

    sns.regplot(x=fitted, y=resid, lowess=True, line_kws={'color':'red'}, scatter_kws={'alpha':alpha}, ax=ax00)
    ax00.set_title('Résidus en fonction des valeurs prédites', size=10)
    ax00.set_xlabel('Valeurs prédites')
    ax00.set_ylabel('Résidus')

    pingouin.qqplot(resid, dist='norm', color='tab:blue', ax=ax01)
    ax01.set_title('Q-Q plot de comparaison à une distribution normale', size=10)
    ax01.set_xlabel('Quantiles de la distribution normale')
    ax01.set_ylabel('Quantiles des résidus')

    sns.regplot(x=fitted, y=resid_sqrt, lowess=True, line_kws={'color':'red'}, scatter_kws={'alpha':alpha}, ax=ax10)
    ax10.set_title('Scale-location', size=10)
    ax10.set_xlabel('Valeurs prédites')
    ax10.set_ylabel('Racine carrée des résidus standardisés')

    sns.regplot(x=leverage, y=resid_sqrt, lowess=True, line_kws={'color':'red'}, scatter_kws={'alpha':alpha}, ax=ax11)
    ax11.set_title('Résidus vs effet de levier', size=10)
    ax11.set_xlabel('Effet de levier')
    ax11.set_ylabel('Résidus standardisés')

    plt.tight_layout()
    plt.show()


In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

model_lin_acc_mois = ols("Effectif ~ C(mois)", data=acc_par_mois).fit()


# Valeurs prédites
fitted_lm = model_lin_acc_mois.fittedvalues
# Résidus
residus_lm = model_lin_acc_mois.resid
#residus standardisés et leur racine carrée
influence_lm = model_lin_acc_mois.get_influence()
residus_norm_lm = influence_lm.resid_studentized_internal
#effet de levier
leverage_lm = influence_lm.hat_matrix_diag

# Graphiques de diagnostic
diagnostic_plots(fitted_lm, residus_lm, residus_norm_lm, leverage_lm)

On constate que les résidus ne suivent pas une distribution normale mais nous prendrons le parti qu'il sont suffisamment proches d'une distribution normale pour réaliser une ANOVA. 

ANOVA :

- Hypothèse nulle $\small H_0$ : le nombre moyen d'accidents est le même suivant le mois.
- Hypothèse alternative $\small H_1$ : le nombre moyen est différent suivant le mois.
- Seuil de signification : $\small\alpha=0.05$

In [ ]:
anova_table = sm.stats.anova_lm(model_lin_acc_mois, typ=2)
anova_table

L'ANOVA permet de rejeter l'hypothèse nulle au risque de 5% : $\small F(11,\,180)=6.37$, $\small p<0.0001^{***}$. On en conclut qu'il y a une différence significative du nombre moyen d'accidents suivant le mois.

### 2.2.3 Test de Kruskal-Wallis

Compte-tenu de la non normalité des résidus du modèle linéaire, un test non paramétrique de Kruskal-Wallis est plus adapté qu'une ANOVA.

- Hypothèse nulle $\small H_0$ : le nombre médian d'accidents est le même suivant le mois.
- Hypothèse alternative $\small H_1$ : le nombre médian est différent suivant le mois.
- Seuil de signification : $\small\alpha=0.05$

In [ ]:
liste_groupes = []
# Pour chaque mois, on crée une liste du nombre d'accidents sur les années 2005-2021 (hors 2020)
for i in range(1,13):
    df = acc_par_mois.loc[acc_par_mois['mois']==i]
    liste_groupes.append(df['Effectif'].tolist())

In [ ]:
from scipy import stats

stats.kruskal(liste_groupes[0], liste_groupes[1], liste_groupes[2],
             liste_groupes[3], liste_groupes[4], liste_groupes[5],
             liste_groupes[6], liste_groupes[7], liste_groupes[8],
             liste_groupes[9], liste_groupes[10], liste_groupes[11])

Le test de Kruskal-Wallis permet de rejeter l'hypothèse $\small H_0$ au risque de 5% : $\chi^2(11)=56.59, p<0.001^{***}$. Le nombre médian d'accidents est différent suivant le mois de l'année.

### 2.2.4. Comparaison multiple
Comparaisons multiples des mois deux à deux.

In [ ]:
import statsmodels.stats.multicomp as multi

multicomp_mois = multi.MultiComparison(acc_par_mois['Effectif'], acc_par_mois['mois'])

# Effectuer le test post-hoc de Tukey
result = multicomp_mois.tukeyhsd()

# Afficher les résultats
print(result)

## 2.3. Y-a-t-il des mois plus meurtriers que d'autres ?
### 2.3.1. Calcul du nombre moyen de morts pour chaque mois de l'année sur la période 2005-2021

In [ ]:
# Nombre de morts par année et par mois
morts_par_mois = (df_usagers.loc[df_usagers['grav']==2, ['Num_Acc']]
                  .merge(df_caract.loc[df_caract['an']!=2020, ['Num_Acc', 'an', 'mois']], how='inner', left_on='Num_Acc', right_on='Num_Acc')
                  .groupby(['an','mois'])
                  .agg(Effectif = pd.NamedAgg('Num_Acc','count'))
                  .reset_index()
                 )
morts_par_mois.head()

In [ ]:
morts_par_mois['Effectif'].describe()

335 morts par mois en moyenne sur la période 2005-2017 (hors année 2020)

In [ ]:
fig, ax= plt.subplots(figsize=(6,4))
sns.histplot(x='Effectif', data=morts_par_mois, bins=40)
plt.xlabel("Nombre de morts")
plt.ylabel("Nombre de mois")
plt.title("Distribution du nombre de morts mensuel")
plt.show()

On remarque un outlier. Regardons à quel mois il correspond.

In [ ]:
morts_par_mois.loc[morts_par_mois['Effectif']==morts_par_mois['Effectif'].max()]

Il s'agit de juillet 2005. Après recherche sur internet, il n'apparaît pas de raison particulière.

In [ ]:
# Moyenne par mois
moy_morts_mois = (morts_par_mois
                .groupby('mois')
                .agg(moyenne = pd.NamedAgg('Effectif','mean'),
                     mediane = pd.NamedAgg('Effectif','median'),
                     variance = pd.NamedAgg('Effectif','var'),
                     std = pd.NamedAgg('Effectif','std'))
                .reset_index()
               )
moy_morts_mois

In [ ]:
# Représentation graphique du nombre d'accidents moyen par mois de l'année

fig, ax = plt.subplots(figsize=(8,4))
sns.barplot(x='mois', y='Effectif', data= morts_par_mois, color='tab:blue', errorbar='sd')
plt.ylim(0,500)
plt.xlabel('Mois', size=12)
plt.ylabel("Nombre moyen de morts", size=12)
plt.xticks([i for i in range(12)], ["Janv", "Fév", "Mars", "Avr", "Mai", "Juin", "Juil", "Août", "Sept", "Oct", "Nov", "Déc"], rotation=45)
plt.grid(visible=True, axis='y', color="#e0e0e0")
plt.title("Nombre moyen de morts et écart-type pour chaque mois de l'année sur la période 2005-2021 (hors 2020)", size=12)
ax.set_axisbelow(True)
plt.show()

In [ ]:
# boxplot du nombre de morts par mois de l'année

fig, ax = plt.subplots(figsize=(8,4))
sns.boxplot(x='mois', y='Effectif', data= morts_par_mois, color='tab:blue', whis=[0,100])
plt.xlabel('Mois', size=12)
plt.ylabel("Nombre de morts", size=12)
plt.xticks([i for i in range(12)], ["Janv", "Fév", "Mars", "Avr", "Mai", "Juin", "Juil", "Août", "Sept", "Oct", "Nov", "Déc"], rotation=45)
plt.grid(visible=True, axis='y', color="#e0e0e0")
plt.title("Diagrammes en boîte du nombre de morts pour chaque mois de l'année sur la période 2005-2021 (hors 2020)", size=12)
ax.set_axisbelow(True)
plt.show()

Résultats intéressants : 
- Alors que le mois de juillet ne se classe que 4ème en nombre moyen d'accidents, c'est le mois le plus meurtrier.
- Le mois d'août est un des mois avec le plus faible nombre moyen d'accidents mais un des plus hauts nombres moyens de morts.

Regardons si les différences observées entre mois sont significatives.

### 2.3.2 ANOVA

In [ ]:
model_lin_morts_mois = ols("Effectif ~ C(mois)", data=morts_par_mois).fit()


# Valeurs prédites
fitted_lm = model_lin_morts_mois.fittedvalues
# Résidus
residus_lm = model_lin_morts_mois.resid
#residus standardisés et leur racine carrée
influence_lm = model_lin_morts_mois.get_influence()
residus_norm_lm = influence_lm.resid_studentized_internal
#effet de levier
leverage_lm = influence_lm.hat_matrix_diag

# Graphiques de diagnostic
diagnostic_plots(fitted_lm, residus_lm, residus_norm_lm, leverage_lm)

Là-encore, les résidus ne suivent pas une loi normale mais nous considérerons que nous pouvons faire une ANOVA :

- Hypothèse nulle $\small H_0$ : le nombre moyen de morts est le même suivant le mois.
- Hypothèse alternative $\small H_1$ : le nombre moyen est différent suivant le mois.
- Seuil de signification : $\small\alpha=0.05$

In [ ]:
anova_table_morts = sm.stats.anova_lm(model_lin_morts_mois, typ=2)
anova_table_morts

L'ANOVA permet de rejeter l'hypothèse nulle au risque de 5% : $\small F(11,\,180)=6.52$, $\small p<0.0001^{***}$. On en conclut qu'il y a une différence significative du nombre moyen de morts suivant le mois.

### 2.3.3. Test de Kruskal-Wallis

- Hypothèse nulle $\small H_0$ : le nombre médian de morts est le même suivant le mois.
- Hypothèse alternative $\small H_1$ : le nombre médian est différent suivant le mois.
- Seuil de signification : $\small\alpha=0.05$

In [ ]:
liste_groupes = []
# Pour chaque mois, on crée une liste du nombre de morts sur les années 2005-2021 (hors 2020)
for i in range(1,13):
    df = morts_par_mois.loc[morts_par_mois['mois']==i]
    liste_groupes.append(df['Effectif'].tolist())

In [ ]:
from scipy import stats

stats.kruskal(liste_groupes[0], liste_groupes[1], liste_groupes[2],
             liste_groupes[3], liste_groupes[4], liste_groupes[5],
             liste_groupes[6], liste_groupes[7], liste_groupes[8],
             liste_groupes[9], liste_groupes[10], liste_groupes[11])

Le test de Kruskal-Wallis permet de rejeter l'hypothèse nulle au risque de 5% : $\small \chi^2(11) = 57.45, p<0.0001^{***}$. Le nombre médian de morts diffère suivant les mois de l'année.

### 2.3.4. Comparaisons multiples

In [ ]:
multicomp_mois = multi.MultiComparison(morts_par_mois['Effectif'], morts_par_mois['mois'])

# Effectuer le test post-hoc de Tukey
result = multicomp_mois.tukeyhsd()

# Afficher les résultats
print(result)

## 2.4. Comparaison du taux de mortalité

Le fait que l'on observe des mois avec "peu d'accidents" et un grand nombre de morts pousse à calculer le taux de mortalité que nous définirons de la manière suivante :

**taux de mortalité = nombre de morts pour 100 accidents**

Nous utiliserons ce calcul plutôt que le pourcentage d'accidents mortels parmi les accidents corporels car il peut y avoir plusieurs morts lors d'un accident.

### 2.4.1. Calcul des taux de mortalité

In [ ]:
#jointure acc_par_mois et morts_par_mois
acc_vs_morts_par_mois = (acc_par_mois
                         .rename(columns={'Effectif':'accidents'})
                         .merge(morts_par_mois.rename(columns={'Effectif':'morts'}), how='inner', on=['an','mois'])
                        )
acc_vs_morts_par_mois.head()

In [ ]:
# Calcul du taux de mortalité
acc_vs_morts_par_mois['tx_mort'] = acc_vs_morts_par_mois['morts']/acc_vs_morts_par_mois['accidents']*100
acc_vs_morts_par_mois.head()

In [ ]:
# Taux de mortalité moyen pour chaque mois de l'année
(acc_vs_morts_par_mois
 .groupby('mois')
 .agg(taux_moyen = pd.NamedAgg('tx_mort','mean'),
      taux_median = pd.NamedAgg('tx_mort','median'),
      variance = pd.NamedAgg('tx_mort','var'),
      std = pd.NamedAgg('tx_mort','std'))
)

In [ ]:
# Représentation graphique du taux de mortalité moyen par mois de l'année

fig, ax = plt.subplots(figsize=(8,4))
sns.barplot(x='mois', y='tx_mort', data= acc_vs_morts_par_mois, color='tab:blue', errorbar='sd')
plt.xlabel('Mois', size=12)
plt.ylabel("Nombre de morts pour 100 accidents", size=12)
plt.xticks([i for i in range(12)], ["Janv", "Fév", "Mars", "Avr", "Mai", "Juin", "Juil", "Août", "Sept", "Oct", "Nov", "Déc"], rotation=45)
plt.grid(visible=True, axis='y', color="#e0e0e0")
plt.title("Taux de mortalité moyen et écart-type pour chaque mois de l'année sur la période 2005-2021 (hors 2020)", size=12)
ax.set_axisbelow(True)
plt.show()

In [ ]:
# boxplot du taux de mortalité par mois de l'année

fig, ax = plt.subplots(figsize=(8,4))
sns.boxplot(x='mois', y='tx_mort', data= acc_vs_morts_par_mois, color='tab:blue', whis=[0,100])
plt.xlabel('Mois', size=12)
plt.ylabel("Nombre de morts pour 100 accidents", size=12)
plt.xticks([i for i in range(12)], ["Janv", "Fév", "Mars", "Avr", "Mai", "Juin", "Juil", "Août", "Sept", "Oct", "Nov", "Déc"], rotation=45)
plt.grid(visible=True, axis='y', color="#e0e0e0")
plt.title("Diagrammes en boîte du taux de mortalité pour chaque mois de l'année sur la période 2005-2021 (hors 2020)", size=12)
ax.set_axisbelow(True)
plt.show()

On voit clairement trois mois avec des taux de mortalité supérieurs aux autres : juillet, août et décembre. On peut avancer les hyptohèses des départs en vacances. Testons si les différences observées sont significatives.

### 2.4.2. ANOVA

In [ ]:
model_lin_tx_mort = ols("tx_mort ~ C(mois)", data=acc_vs_morts_par_mois).fit()


# Valeurs prédites
fitted_lm = model_lin_tx_mort.fittedvalues
# Résidus
residus_lm = model_lin_tx_mort.resid
#residus standardisés et leur racine carrée
influence_lm = model_lin_tx_mort.get_influence()
residus_norm_lm = influence_lm.resid_studentized_internal
#effet de levier
leverage_lm = influence_lm.hat_matrix_diag

# Graphiques de diagnostic
diagnostic_plots(fitted_lm, residus_lm, residus_norm_lm, leverage_lm)

Mis à part deux résidus, les résidus suivent approximativement une loi normale et l'homoscédasticité est relativement bonne. Nous considèrerons que nous sommes dans des conditions valides d'utilisation d'une ANOVA

- Hypothèse nulle $\small H_0$ : le taux de mortalité est le même suivant le mois.
- Hypothèse alternative $\small H_1$ : le taux de mortalité est différent suivant le mois.
- Seuil de signification : $\small\alpha=0.05$

In [ ]:
anova_tx_mort = sm.stats.anova_lm(model_lin_tx_mort, typ=2)
anova_tx_mort

L'ANOVA permet de rejeter l'hypothèse nulle : $\small F(11, 180) = 23.18, p<0.000^{***}$. On en déduit qu'il y a une différence significative entre les taux de mortalités des différents mois de l'année. Le text de Tuckey devrait nous montrer qu'il y a une déifférence significative entre les mois de juillet, août et décembre d'un côté, et les autres mois de l'autre côté.

### 2.4.3. Test de Tuckey

In [ ]:
multicomp_mois = multi.MultiComparison(acc_vs_morts_par_mois['tx_mort'], acc_vs_morts_par_mois['mois'])

# Effectuer le test post-hoc de Tukey
result = multicomp_mois.tukeyhsd()

# Afficher les résultats
print(result)

- Le taux de mortalité de juillet est significativement différent de celui de tous les autres mois de l'année à l'exception de août et décembre ($\small p<0.0001^{***}$ à chaque fois)
- Le taux de mortalité d'août est significativement différent de celui de tous les autres mois de l'année à l'exception de juillet uniquement ($\small p<0.0001^{***}$ à chaque fois)
- Le taux de mortalité de décembre est significativement différent de celui de tous les autres mois de l'année à l'exception de juillet, août et février.

## 2.5. Taux de mortalité suivant le jour de la semaine.

### 2.5.1 Calcul du taux de mortalité chaque jour

Les taux de mortalité élevés aux mois de juillet, août et décembre amènent à se poser la question de leur cause. Une hyptohèse est un taux de mortalité élevé lors des week-ends de départ en vacances. Mais il faut aussi regarder les taux de mortalités le week-end les autres mois. En effet, si on observe une élévation du taux de mortalité le week-end en juillet et août mais également les autres mois, celà ne permettra pas de conclure en l'influence des week-ends de départ en vacances.

Pour étudier ces taux, il faut créer une variable date et y faire correspondre le jour de la semaine.



In [ ]:
# Création de la variable date
df_caract['date'] = pd.to_datetime(df_caract[['an','mois','jour']].rename(columns={'an':'year', 'mois':'month', 'jour':'day'}))
df_caract[['an','mois','jour','date']].head()

In [ ]:
# Création du jour de la semaine
df_caract['nom_jour'] = df_caract['date'].dt.day_name(locale='fr_FR.utf8')
df_caract['nom_mois'] = df_caract['date'].dt.month_name(locale='fr_FR.utf8')
df_caract[['an','mois','jour','date','nom_jour', 'nom_mois']].head()

In [ ]:
acc_par_jour = (df_caract
                .groupby(['an','mois','nom_mois','nom_jour'])
                .agg(accidents = pd.NamedAgg('Num_Acc','count'))
                .reset_index()
                )
acc_par_jour.head()

In [ ]:
morts_par_jour = (df_caract[['Num_Acc','an','mois','jour','date','nom_jour']]
                  .merge(df_usagers.loc[df_usagers['grav']==2], on='Num_Acc', how='inner')
                  .groupby(['an','mois','nom_jour'])
                  .agg(morts = pd.NamedAgg('Num_Acc','count'))
                  .reset_index()
                 )
morts_par_jour.head()

Nous pouvons maintenant étudier le taux de mortalité pour chaque jour de la semaine et pour chaque mois de l'année.

In [ ]:
acc_vs_morts_par_jour = acc_par_jour.merge(morts_par_jour, on=['an','mois','nom_jour'], how='inner')
acc_vs_morts_par_jour['tx_mortalite'] = acc_vs_morts_par_jour['morts']/acc_vs_morts_par_jour['accidents']*100
acc_vs_morts_par_jour.head()

In [ ]:
acc_vs_morts_pivot = round(acc_vs_morts_par_jour.pivot_table(index='mois', columns='nom_jour', values='tx_mortalite', aggfunc='mean'), 1)
acc_vs_morts_pivot

In [ ]:
ordre_jours = ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche']

g = sns.catplot(x='nom_jour', y='tx_mortalite', data=acc_vs_morts_par_jour, 
                kind='box', order=ordre_jours, whis=[0,100], color='tab:blue',
                col='nom_mois', col_wrap=4,
                height=3, aspect=1)

plt.xlabel('Jour de la semaine')
plt.ylabel('Nombre de morts pour 100 accidents')
plt.suptitle("Taux de mortalité par jour de la semaine pour chaque mois de l'année sur la période 2005-2021", size=12, y=1.02)
g.set_titles('{col_name}',size=10)
g.set_xlabels("Jour de la semaine", size=10)
g.set_xticklabels(['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche'], rotation=45)
g.set_ylabels("Nombre de morts pour 100 accidents",size=10)
g.fig.subplots_adjust(hspace = 1)
plt.tight_layout()
plt.show()

Quel que soit le mois, le taux de mortalité augmente le week-end. Quel différence y-a-t-il entre le taux de mortalité moyen le week-end et en semaine pour chaque mois ?

### 2.5.2 Comparaison des taux de mortalités en semaine et le week-end pous chaque mois de l'année.

In [ ]:
tx_mort_we = (acc_vs_morts_par_jour
              .loc[acc_vs_morts_par_jour['nom_jour'].isin(['Samedi', 'Dimanche'])]
              .groupby(['mois','nom_mois'])
              .agg(tx_mort_we = pd.NamedAgg('tx_mortalite','mean'))
              .reset_index()
             )
tx_mort_we

In [ ]:
tx_mort_sem = (acc_vs_morts_par_jour
              .loc[~acc_vs_morts_par_jour['nom_jour'].isin(['Samedi', 'Dimanche'])]
              .groupby(['mois','nom_mois'])
              .agg(tx_mort_sem = pd.NamedAgg('tx_mortalite','mean'))
              .reset_index()
             )
tx_mort_sem

In [ ]:
comp_tx_mort_we_sem = tx_mort_we.merge(tx_mort_sem, on=['mois','nom_mois'])
comp_tx_mort_we_sem['diff'] = comp_tx_mort_we_sem['tx_mort_we']-comp_tx_mort_we_sem['tx_mort_sem']
comp_tx_mort_we_sem

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))
sns.barplot(x='nom_mois', y='diff', data=comp_tx_mort_we_sem, color='tab:blue', ax=ax)
plt.title("Différence entre le taux de mortalité moyen par jour en week-end et en semaine suivant le mois de l'année sur la période 2005-2021", size=12)
plt.xlabel('Mois')
plt.ylabel('Différence')
plt.xticks(rotation=45)
plt.show()

On n'observe pas une différence supérieure en juillet et août concernant du taux de mortalité entre le week-end et la semaine. Le taux de mortalité supérieur de ces deux mois ne s'explique donc pas par les accidents des week-ends de départ en vacance. Le taux est supérieur sur l'ensemble de la semaine.

### 2.5.3 La différence entre le taux de mortalité en semaine et le week-end est-elle significative ?

In [ ]:
# Création d'une variable week-end : 1 pour week-end et 0 pour semaine

acc_vs_morts_par_jour['weekend'] = acc_vs_morts_par_jour['nom_jour'].map(lambda x : 1 if x in ['Samedi', 'Dimanche'] else 0)

acc_vs_morts_par_jour.head()

In [ ]:
#modèle linéaire du taux de mortalité en fonction de la variable week-end
mod_lin_tx_mort_vs_we = ols('tx_mortalite ~ C(weekend)', data=acc_vs_morts_par_jour).fit()


# Valeurs prédites
fitted_lm = mod_lin_tx_mort_vs_we.fittedvalues
# Résidus
residus_lm = mod_lin_tx_mort_vs_we.resid
#residus standardisés et leur racine carrée
influence_lm = mod_lin_tx_mort_vs_we.get_influence()
residus_norm_lm = influence_lm.resid_studentized_internal
#effet de levier
leverage_lm = influence_lm.hat_matrix_diag

# Graphiques de diagnostic
np.seterr(invalid='ignore')
diagnostic_plots(fitted_lm, residus_lm, residus_norm_lm, leverage_lm)

Les résidus suivent approximativement une loi normale. On ne remarque pas non plus de problème sur le graphique des résidus en fonction des valeurs prédites. Exécutons une ANOVA :

- Hypothèse nulle $\small H_0$ : le taux de mortalité moyen par jour est le même en week-end et en semaine.
- Hypothèse alternative $\small H_1$ : le taux de mortalité moyen par jour est différent en week-end et en semaine.
- Seuil de signification : $\small\alpha=0.05$

In [ ]:
anova = sm.stats.anova_lm(mod_lin_tx_mort_vs_we)
anova

On observe une différence très significative entre le taux de mortalité moyen en week-end et en semaine : $\small F(1, 1426)=1415.1, p<0.0001^{***}$. Le taux de mortalité est supérieur le week-end.